# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 2 2022 9:41AM,249863,16,SHL-8724024,"SHL Pharma, LLC",Released
1,Nov 2 2022 9:38AM,249862,15,VT80179936,"Virtus Pharmaceuticals, LLC",Released
2,Nov 2 2022 9:38AM,249862,15,VT80179937,"Virtus Pharmaceuticals, LLC",Released
3,Nov 2 2022 9:38AM,249862,15,VT80180014,"Virtus Pharmaceuticals, LLC",Released
4,Nov 2 2022 9:38AM,249862,15,VT80180015,"Virtus Pharmaceuticals, LLC",Released
5,Nov 2 2022 9:38AM,249862,15,VT80180016,"Virtus Pharmaceuticals, LLC",Released
6,Nov 2 2022 9:38AM,249862,15,VT80180017,"Virtus Pharmaceuticals, LLC",Released
7,Nov 2 2022 9:38AM,249862,15,VT80180018,"Virtus Pharmaceuticals, LLC",Released
8,Nov 2 2022 9:38AM,249862,15,VT80180019,"Virtus Pharmaceuticals, LLC",Released
9,Nov 2 2022 9:38AM,249862,15,VT80180020,"Virtus Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
23,249859,Released,1
24,249860,Released,19
25,249861,Released,8
26,249862,Released,15
27,249863,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
249859,NaN,1.0
249860,NaN,19.0
249861,NaN,8.0
249862,NaN,15.0
249863,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
249793,1.0,0.0
249794,0.0,1.0
249795,0.0,1.0
249807,6.0,9.0
249828,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
249793,1,0
249794,0,1
249795,0,1
249807,6,9
249828,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,249793,1,0
1,249794,0,1
2,249795,0,1
3,249807,6,9
4,249828,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,249793,1,
1,249794,,1
2,249795,,1
3,249807,6,9
4,249828,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 2 2022 9:41AM,249863,16,"SHL Pharma, LLC"
1,Nov 2 2022 9:38AM,249862,15,"Virtus Pharmaceuticals, LLC"
16,Nov 2 2022 9:35AM,249861,15,"Mizner Bioscience, LLC"
24,Nov 2 2022 9:27AM,249858,10,Eywa Pharma Inc.
36,Nov 2 2022 9:26AM,249860,10,ISDIN Corporation
55,Nov 2 2022 9:26AM,249857,15,"Brookfield Pharmaceuticals, LLC"
83,Nov 2 2022 9:22AM,249859,10,Eywa Pharma Inc.
84,Nov 2 2022 9:13AM,249855,10,ISDIN Corporation
97,Nov 2 2022 9:09AM,249854,10,ISDIN Corporation
134,Nov 2 2022 9:05AM,249853,19,ACG North America LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Nov 2 2022 9:41AM,249863,16,"SHL Pharma, LLC",,1
1,Nov 2 2022 9:38AM,249862,15,"Virtus Pharmaceuticals, LLC",,15
2,Nov 2 2022 9:35AM,249861,15,"Mizner Bioscience, LLC",,8
3,Nov 2 2022 9:27AM,249858,10,Eywa Pharma Inc.,,12
4,Nov 2 2022 9:26AM,249860,10,ISDIN Corporation,,19
5,Nov 2 2022 9:26AM,249857,15,"Brookfield Pharmaceuticals, LLC",,28
6,Nov 2 2022 9:22AM,249859,10,Eywa Pharma Inc.,,1
7,Nov 2 2022 9:13AM,249855,10,ISDIN Corporation,,13
8,Nov 2 2022 9:09AM,249854,10,ISDIN Corporation,,37
9,Nov 2 2022 9:05AM,249853,19,ACG North America LLC,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 2 2022 9:41AM,249863,16,"SHL Pharma, LLC",1,
1,Nov 2 2022 9:38AM,249862,15,"Virtus Pharmaceuticals, LLC",15,
2,Nov 2 2022 9:35AM,249861,15,"Mizner Bioscience, LLC",8,
3,Nov 2 2022 9:27AM,249858,10,Eywa Pharma Inc.,12,
4,Nov 2 2022 9:26AM,249860,10,ISDIN Corporation,19,
5,Nov 2 2022 9:26AM,249857,15,"Brookfield Pharmaceuticals, LLC",28,
6,Nov 2 2022 9:22AM,249859,10,Eywa Pharma Inc.,1,
7,Nov 2 2022 9:13AM,249855,10,ISDIN Corporation,13,
8,Nov 2 2022 9:09AM,249854,10,ISDIN Corporation,37,
9,Nov 2 2022 9:05AM,249853,19,ACG North America LLC,3,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 2 2022 9:41AM,249863,16,"SHL Pharma, LLC",1,
1,Nov 2 2022 9:38AM,249862,15,"Virtus Pharmaceuticals, LLC",15,
2,Nov 2 2022 9:35AM,249861,15,"Mizner Bioscience, LLC",8,
3,Nov 2 2022 9:27AM,249858,10,Eywa Pharma Inc.,12,
4,Nov 2 2022 9:26AM,249860,10,ISDIN Corporation,19,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Nov 2 2022 9:41AM,249863,16,"SHL Pharma, LLC",1.0,NaN
1,Nov 2 2022 9:38AM,249862,15,"Virtus Pharmaceuticals, LLC",15.0,NaN
2,Nov 2 2022 9:35AM,249861,15,"Mizner Bioscience, LLC",8.0,NaN
3,Nov 2 2022 9:27AM,249858,10,Eywa Pharma Inc.,12.0,NaN
4,Nov 2 2022 9:26AM,249860,10,ISDIN Corporation,19.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 2 2022 9:41AM,249863,16,"SHL Pharma, LLC",1.0,0.0
1,Nov 2 2022 9:38AM,249862,15,"Virtus Pharmaceuticals, LLC",15.0,0.0
2,Nov 2 2022 9:35AM,249861,15,"Mizner Bioscience, LLC",8.0,0.0
3,Nov 2 2022 9:27AM,249858,10,Eywa Pharma Inc.,12.0,0.0
4,Nov 2 2022 9:26AM,249860,10,ISDIN Corporation,19.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2248659,110.0,11.0
12,249849,22.0,0.0
15,999417,53.0,0.0
16,249863,1.0,0.0
19,1249178,17.0,8.0
21,749382,2.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2248659,110.0,11.0
1,12,249849,22.0,0.0
2,15,999417,53.0,0.0
3,16,249863,1.0,0.0
4,19,1249178,17.0,8.0
5,21,749382,2.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,110.0,11.0
1,12,22.0,0.0
2,15,53.0,0.0
3,16,1.0,0.0
4,19,17.0,8.0
5,21,2.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,110.0
1,12,Released,22.0
2,15,Released,53.0
3,16,Released,1.0
4,19,Released,17.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21
Status,,,,,,
Executing,11.0,0.0,0.0,0.0,8.0,1.0
Released,110.0,22.0,53.0,1.0,17.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21
0,Executing,11.0,0.0,0.0,0.0,8.0,1.0
1,Released,110.0,22.0,53.0,1.0,17.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21
0,Executing,11.0,0.0,0.0,0.0,8.0,1.0
1,Released,110.0,22.0,53.0,1.0,17.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()